In [62]:
message = '''
Instrument : XAUUSD
Side : Buy
Entry Level : 2431
Stop Loss :  2423
Take Profit : 2440
Relevance : 10 hours
Confidence : low
Date/Time : 09.08.2024 15:20
Source: My_sorce

Description: TEST_2
'''

In [63]:
from datetime import datetime, timedelta
import pandas as pd

In [64]:
def make_dict(message:str) -> dict:
  lines = [line.strip() for line in message.strip().split('\n') if line.strip()]
  data_dict = {}
  for line in lines:
      if ':' in line:
          key, value = line.split(':', 1)
          data_dict[key.strip()] = value.strip()
  data_dict['headlinear'] = data_dict['Instrument'] + ' ' + data_dict['Side'] +  ' Signal'
  data_dict['Description'] = '\n' + data_dict['Description']
  return data_dict

In [65]:
data_dict = make_dict(message)
data_dict

{'Instrument': 'XAUUSD',
 'Side': 'Buy',
 'Entry Level': '2431',
 'Stop Loss': '2423',
 'Take Profit': '2440',
 'Relevance': '10 hours',
 'Confidence': 'low',
 'Date/Time': '09.08.2024 15:20',
 'Source': 'My_sorce',
 'Description': '\nTEST_2',
 'headlinear': 'XAUUSD Buy Signal'}

In [66]:
key_data = pd.read_csv('IDes for Unity Instruments.csv')

def make_correct_time(time:str, delta:int=0) -> str:
    date_obj = datetime.strptime(time, '%d.%m.%Y %H:%M')
    date_obj += timedelta(hours=delta)
    formatted_date = date_obj.strftime('%Y-%m-%dT%H:%M:%SZ')
    return formatted_date

target = {
"instrumentIds": [int(i) for i in key_data.loc[key_data['ticker'] == data_dict.get('Instrument', None)]['id'].iloc[0].split(',')],
"openOrderType": "LIMIT",
"side": (data_dict['Side']).upper(),
"price": float(data_dict['Entry Level'].replace(',', '.')),
"stopLoss": float(data_dict['Stop Loss'].replace(',', '.')),
"takeProfit": float(data_dict['Take Profit'].replace(',', '.')),
"confidence": 0,
"expirationTime": make_correct_time(time=data_dict['Date/Time'], delta=int(data_dict['Relevance'].split()[0])),
"localeDescriptions": [
    {
    "locale": 'EN',
    "target": data_dict['headlinear'],
    "background": data_dict['Description']
    }
],
"publishTime": make_correct_time(time=data_dict['Date/Time']),
"state": "LIVE_TRADE",
"provider": "UNITY",
"rate": 0,
"externalId": "string",
"sorce": (data_dict['Source']),
# "sorce": 'TKoF',
"priority": 0
}

target

{'instrumentIds': [20225, 59534, 48380, 59618],
 'openOrderType': 'LIMIT',
 'side': 'BUY',
 'price': 2431.0,
 'stopLoss': 2423.0,
 'takeProfit': 2440.0,
 'confidence': 0,
 'expirationTime': '2024-08-10T01:20:00Z',
 'localeDescriptions': [{'locale': 'EN',
   'target': 'XAUUSD Buy Signal',
   'background': '\nTEST_2'}],
 'publishTime': '2024-08-09T15:20:00Z',
 'state': 'LIVE_TRADE',
 'provider': 'UNITY',
 'rate': 0,
 'externalId': 'string',
 'sorce': 'My_sorce',
 'priority': 0}

In [67]:
target['instrumentIds'] = [20225]
target

{'instrumentIds': [20225],
 'openOrderType': 'LIMIT',
 'side': 'BUY',
 'price': 2431.0,
 'stopLoss': 2423.0,
 'takeProfit': 2440.0,
 'confidence': 0,
 'expirationTime': '2024-08-10T01:20:00Z',
 'localeDescriptions': [{'locale': 'EN',
   'target': 'XAUUSD Buy Signal',
   'background': '\nTEST_2'}],
 'publishTime': '2024-08-09T15:20:00Z',
 'state': 'LIVE_TRADE',
 'provider': 'UNITY',
 'rate': 0,
 'externalId': 'string',
 'sorce': 'My_sorce',
 'priority': 0}

In [68]:
import requests
import json

In [69]:
payload =  json.dumps(target, indent=4, ensure_ascii=False)

print(payload)

{
    "instrumentIds": [
        20225
    ],
    "openOrderType": "LIMIT",
    "side": "BUY",
    "price": 2431.0,
    "stopLoss": 2423.0,
    "takeProfit": 2440.0,
    "confidence": 0,
    "expirationTime": "2024-08-10T01:20:00Z",
    "localeDescriptions": [
        {
            "locale": "EN",
            "target": "XAUUSD Buy Signal",
            "background": "\nTEST_2"
        }
    ],
    "publishTime": "2024-08-09T15:20:00Z",
    "state": "LIVE_TRADE",
    "provider": "UNITY",
    "rate": 0,
    "externalId": "string",
    "sorce": "My_sorce",
    "priority": 0
}


In [70]:
url = 'https://rest.unity.finance/api/v1/createTradeIdea'
token = 'c3f28d5b-9694-4307-b30f-6f800ee13506'
headers = {
    "accept": "application/json",
    "auth-token": token
}
response = requests.post(url, headers=headers, data=(payload))
response

<Response [200]>